In [64]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("CH12-RAG")



LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


In [45]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


# 단계 1: 문서 로드(Load Documents)
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
url = "https://n.news.naver.com/article/437/0000378416"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)
docs = loader.load()


# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(docs)

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "부영그룹의 출산 장려 정책에 대해 설명해주세요"
response = rag_chain.invoke(question)

# 결과 출력
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

URL: https://n.news.naver.com/article/437/0000378416
문서의 수: 1
[HUMAN]
부영그룹의 출산 장려 정책에 대해 설명해주세요

[AI]
부영그룹은 출산을 장려하기 위해 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다. 해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 또한, 셋째까지 낳는 경우에는 국민주택을 제공할 예정입니다.


In [46]:
# Web

loader = WebBaseLoader(
    web_paths = ("https://www.bbc.com/news/business-68092814",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "main",
            attrs={"id": ["main-content"]},
        )
    ),
)

docs = loader.load()

print(f"문서의 수 : {len(docs)}")
docs[0].page_content[:500]


문서의 수 : 1


'Could AI \'trading bots\' transform the world of investing?Getty ImagesIt is hard for both humans and computers to predict stock market movementsSearch for "AI investing" online, and you\'ll be flooded with endless offers to let artificial intelligence manage your money.I recently spent half an hour finding out what so-called AI "trading bots" could apparently do with my investments.Many prominently suggest that they can give me lucrative returns. Yet as every reputable financial firm warns - your '

In [47]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드, 파일의 경로 입력
loader = PyPDFLoader("12-RAG/data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 페이지 별 문서 로드
docs = loader.load()

print(f"\n[페이지내용]\n{docs[5].page_content[:500]}")
print(f"\n[metadata]\n{docs[5].metadata}\n")



 17%|█▋        | 1/6 [39:37<3:18:09, 2377.97s/it]



[페이지내용]
1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육
영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언
n영국 블레츨리 파크에서 개최된 AI 안전성 정상회의에 참가한 28개국들이 AI 안전 보장을 
위한 협력 방안을 담은 블레츨리 선언을 발표
n첨단 AI를 개발하는 국가와 기업들은 AI 시스템에 대한 안전 테스트 계획에 합의했으며 , 
영국의 AI 안전 연구소가 전 세계 국가와 협력해 테스트를 주도할 예정 KEY Contents
£AI 안전성 정상회의 참가국들 , 블레츨리 선언 통해 AI 안전 보장을 위한 협력에 합의
n2023 년 11월 1~2일 영국 블레츨리 파크에서 열린 AI 안전성 정상회의 (AI Safety Summit) 에 
참가한 28개국 대표들이  AI 위험 관리를 위한 ‘블레츨리 선언’을 발표 
∙선언은 AI 안전 보장을 위해 국가, 국제기구 , 기업, 시민사회 , 학계를 포함한 모든 이해관계자의 협력이 
중요하다고 강조했으며 

[metadata]
{'source': '12-RAG/data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 5}



In [48]:
# CSV

from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="12-RAG/data/titanic.csv")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

print(f"\n[페이지내용]\n {docs[5].page_content[:500]}")
print(f"\n[metadata]\n {docs[5].metadata}\n")

문서의 수: 891

[페이지내용]
 PassengerId: 6
Survived: 0
Pclass: 3
Name: Moran, Mr. James
Sex: male
Age: 
SibSp: 0
Parch: 0
Ticket: 330877
Fare: 8.4583
Cabin: 
Embarked: Q

[metadata]
 {'source': '12-RAG/data/titanic.csv', 'row': 5}



In [49]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("12-RAG/data/appendix-keywords.txt",encoding="UTF-8")
docs = loader.load()
print(f"문서의 수 : {len(docs)}")

print(f"\n[페이지내용]\n {docs[0].page_content[:500]}")
print(f"\n[metadata]\n {docs[0].metadata}\n")


문서의 수 : 1

[페이지내용]
 Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어

[metadata]
 {'source': '12-RAG/data/appendix-keywords.txt'}



In [50]:
from langchain_community.document_loaders import DirectoryLoader


# text_loader_kwargs = {'autodetect_encoding': True}
loader = DirectoryLoader(".", glob="12-RAG/data/*.txt",show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")

  0%|          | 0/6 [00:00<?, ?it/s]ERROR:langchain_community.document_loaders.directory:Error loading file 12-RAG\data\appendix-keywords-CP949.txt


ImportError: failed to find libmagic.  Check your installation

In [ ]:
# pdf 로딩
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="12-RAG/data/*.pdf")
docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[2500:3000])

Error loading file 12-RAG\data\SPRI_AI_Brief_2023년12월호_F.pdf


ImportError: unstructured package not found, please install it with `pip install unstructured`

In [ ]:
from langchain_community.document_loaders import PythonLoader

loader = DirectoryLoader(".", glob="12-RAG/data/**/*.py",loader_cls=PythonLoader)
docs = loader.load()

print(f"문서의 수 : {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========================[앞부분] 미리보기 ===================================")
print(docs[0].page_content[:500])

문서의 수 : 1

[메타데이터]

{'source': '12-RAG\\data\\audio_utils.py'}

========================[앞부분] 미리보기 ===================================
import re
import os
from pytube import YouTube
from moviepy.editor import AudioFileClip, VideoFileClip
from pydub import AudioSegment
from pydub.silence import detect_nonsilent


def extract_abr(abr):
    youtube_audio_pattern = re.compile(r"\d+")
    kbps = youtube_audio_pattern.search(abr)
    if kbps:
        kbps = kbps.group()
        return int(kbps)
    else:
        return 0


def get_audio_filepath(filename):
    # audio 폴더가 없으면 생성
    if not os.path.isdir("audio"):
        os.mkdir("au


| MODEL                  | ROUGH PAGES PER DOLLAR | EXAMPLE PERFORMANCE ON MTEB EVAL |
| ---------------------- | ---------------------- | -------------------------------- |
| text-embedding-3-small | 62,500                 | 62.3%                            |
| text-embedding-3-large | 9,615                  | 64.6%                            |
| text-embedding-ada-002 | 12,500                 | 61.0%                            |


In [ ]:
# 유료 과금 임베딩

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
vectorstore = FAISS.from_documents(
    documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

## 오픈소스 임베딩


In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS


vectorstore = FAISS.from_documents(
    documents=splits, embedding=HuggingFaceBgeEmbeddings()
)

c:\Users\UserK\Desktop\zeroOne\ragenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.vectorstores import FAISS

# 예제 문서 리스트
documents = [
    {"text": "This is a sample document."},
    {"text": "Another example document."}
]

# FastEmbed 임베딩 생성
embedding = FastEmbedEmbeddings()

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=documents, embedding=embedding)


Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


ValidationError: 1 validation error for FastEmbedEmbeddings
_model
  extra fields not permitted (type=value_error.extra)

In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

vectorstore = FAISS.from_documents(documents=splits, embedding=FastEmbedEmbeddings())

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

## Retriever 생성

In [ ]:
query = "회사의 저출생 정책이 뭐야?"

retriver = vectorstore.as_retriever(
    search_type="similarity"
    )
search_result = retriver.get_relevant_documents(query)
print(search_result)

c:\Users\UserK\Desktop\zeroOne\ragenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책"), Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content='남성 직원의 육아휴직을 의무화한 곳도 있습니다.사내 어린이집을 밤 10시까지 운영하고 셋째를 낳으면 무조건 승진시켜 주기도 합니다.한 회사는 지난해 네쌍둥이를 낳은 직원에 의료비를 지원해 관심을 모았습니다.정부 대신 회사가 나서는 출산장려책이 사회적 분위기를 바꿀 거라는 기대가 커지는 가운데, 여력이 부족한 중소지원이 필요하다는 목소리도 나옵니다.[영상디자인 곽세미]'), Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.

In [ ]:
# 유사도 기반 검색에서 score_threshold 이상인 결과만 반환한다.

query = "회사의 저출생 정책이 뭐야?"

retriver = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold":0.8}
)

search_result = retriver.get_relevant_documents(query)
print(search_result)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책")]


In [ ]:
query = "회사의 저출생 정책이 뭐야?"

retriver = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":2})

search_result = retriver.get_relevant_documents(query)
print(search_result)

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책"), Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content='남성 직원의 육아휴직을 의무화한 곳도 있습니다.사내 어린이집을 밤 10시까지 운영하고 셋째를 낳으면 무조건 승진시켜 주기도 합니다.한 회사는 지난해 네쌍둥이를 낳은 직원에 의료비를 지원해 관심을 모았습니다.정부 대신 회사가 나서는 출산장려책이 사회적 분위기를 바꿀 거라는 기대가 커지는 가운데, 여력이 부족한 중소지원이 필요하다는 목소리도 나옵니다.[영상디자인 곽세미]')]


## 다양한 쿼리 생성

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

from langchain_openai import ChatOpenAI

query = "회사의 저출생 정책이 뭐야?"

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [ ]:
import logging

# 쿼리가 세그멘테이션이 돼
# 로그를 노출한다.

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

unique_docs =retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 부영그룹이 채택한 출산 장려 정책은 무엇인가요?', '2. 부영그룹의 출산 촉진 정책은 어떻게 운영되고 있나요?', '3. 부영그룹이 시행하는 출산 지원 정책에 대해 자세히 알려주세요.']


In [ ]:
# 앙상블 리트리버?

from langchain.retrievers import BM25Retriever, EnsembleRetriever

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

doc_list = [
    # "난 오늘 잠을 많이 못잤다.",
    # "과제를 내일까지 끝낼 수 있나?",
    # "내가 제일 좋아하는 과목은 파이썬,sql,flask이다.",
    "난 오늘 많이 먹어서 배가 정말 부르다",
    "떠나는 저 배가 오늘 마지막 배인가요?",
    "내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.",
]

bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

faiss_vectorstore = FAISS.from_texts(doc_list,OpenAIEmbeddings())

faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k":2})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5,0.5]
)

In [ ]:
def pretty_print(docs):
    for i, doc in enumerate(docs):
        print(f"[{i+1}] {doc.page_content}")

In [ ]:
sample_query = "난 요즘 피로가 많아졌어...."
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)

print("[BM25 Retriever]")
pretty_print(relevant_docs)

print("==="*20)

relevant_docs = faiss_retriever.get_relevant_documents(sample_query)

print("[FAISS Retriever]")
pretty_print(relevant_docs)

print("==="*20)

relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)

print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
난 요즘 피로가 많아졌어....

[BM25 Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[FAISS Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[3] 떠나는 저 배가 오늘 마지막 배인가요?


In [ ]:
def pretty_print(docs):
    for i, doc in enumerate(docs):
        print(f"[{i+1}] {doc.page_content}")

In [ ]:
sample_query = "나 요즘 배에 정말 살이 많이 쪘어..."
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)

print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)

relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
나 요즘 배에 정말 살이 많이 쪘어...

[BM25 Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[FAISS Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[3] 떠나는 저 배가 오늘 마지막 배인가요?


In [ ]:
sample_query = "바다 위에 떠다니는 배들이 많다"
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
바다 위에 떠다니는 배들이 많다

[BM25 Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[FAISS Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 떠나는 저 배가 오늘 마지막 배인가요?
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[3] 난 오늘 많이 먹어서 배가 정말 부르다


In [ ]:
sample_query = "ships"
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
ships

[BM25 Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[FAISS Retriever]
[1] 떠나는 저 배가 오늘 마지막 배인가요?
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[Ensemble Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?


In [ ]:
sample_query = "pear"
print(f"[Query]\n{sample_query}\n")
relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
pear

[BM25 Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[FAISS Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?


In [53]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback


prompt = hub.pull("rlm/rag-prompt")
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

with get_openai_callback() as cb:
    result = model.invoke("DMC 첨단 산업센터의 위치는 어디인가요?")
    
print(cb)

Tokens Used: 70
	Prompt Tokens: 31
	Completion Tokens: 39
Successful Requests: 1
Total Cost (USD): $7.4e-05


In [65]:
from langchain.llms import HuggingFaceHub

repo_id = "google/flan-t5-xxl"

t5_model = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.1, "max_length": 512}
)

In [68]:
t5_model.invoke("Where is the capital of South korea?")

HfHubHTTPError: 504 Server Error: Gateway Timeout for url: https://api-inference.huggingface.co/models/google/flan-t5-xxl (Request ID: Ej6htaTKfL76bcuLtmjbP)

Model google/flan-t5-xxl time out

In [87]:
# 단계 1: 문서 로드(Load Documents)
# 문서를 로드하고, 청크로 나누고, 인덱싱합니다.
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
file_path = "국방백서.pdf"
loader = PyPDFLoader(file_path=file_path)

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = loader.load_and_split(text_splitter=text_splitter)

# 단계 3, 4: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 5: 리트리버 생성(Create Retriever)
# 사용자의 질문(query) 에 부합하는 문서를 검색합니다.

# 유사도 높은 K 개의 문서를 검색합니다.
k = 3

# (Sparse) bm25 retriever and (Dense) faiss retriever 를 초기화 합니다.
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = k

faiss_vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 7: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 8: 체인 생성(Create Chain)
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "2022년도 군의 전력 현황을 알려주세요."

response = rag_chain.invoke(question)

# 결과 출력
print(f"PDF Path: {file_path}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

PDF Path: 국방백서.pdf
문서의 수: 1
[HUMAN]
2022년도 군의 전력 현황을 알려주세요.

[AI]
2022년 남북 군사력 현황은 한국이 육군 36.5만여명, 해군 7만여명, 공군 6.5만여명 등으로 총 50만여명이며, 북한은 128만여명으로 나타났습니다. 군사력은 양적 비교만으로 제시되었으며, 실질적인 비교를 위해서는 장비성능, 훈련수준, 운용개념 등을 종합적으로 고려해야 합니다. 북한의 특수작전군 병력은 육군, 해군, 공군 등에 포함되어 있습니다.


In [88]:
question = "주요 군사 현황에 대해 설명해주세요"
response = rag_chain.invoke(question)
print(response)

주요 군수물자 생산 및 비축시설은 전시동원체계를 갖추고 있으며, 국제사회의 대북 제재로 인해 군수산업 육성에 어려움이 있을 것으로 평가된다. 한반도 안보와 밀접한 관계가 있는 주요 6개국의 군사력은 상호보완적 동맹관계로 발전하였다. 한국은 한미 군사동맹관계를 통해 군사 및 경제 지원을 받아오며, 국군은 전투능력을 갖추고 있다.


In [89]:
question = "미래의 국방 전망은 어떻게 되나요?"
response = rag_chain.invoke(question)
print(response)

미래의 국방 전망은 첨단전력 건설과 방산수출 확대를 통해 국방력을 강화하고 국가 경제발전에 기여하는 국가전략산업으로 거듭날 수 있을 것이다. 정부는 국가안보전략을 통해 남북관계 정상화와 한반도의 평화를 안착시키고 지구촌 번영에 기여할 것이다. 현재의 국방전략은 외부의 군사적 위협과 침략으로부터 국가를 보위하고 평화통일을 뒷받침하며 지역의 안정과 세계평화에 기여하는 것을 목표로 하고 있다.
